# sklearn源码阅读
___

## 1. 源码基本结构

- `_check_build`

检查是否正确编译的脚本。

- `_build_utils`

官方构建源码时的工具。

- `_loss`

GLM算法会用到的分布函数。

- `cluster`

聚类算法的实现，含有Birch/DBSCAN/HCluster/KMeans/Spectral等算法，其中kmeans算法提供elkan/fast/lloyd等几种用cpython实现的方法。

- `compose`

合成模型使用的元学习器（meta-learner)

- `covariance`

计算特征之间的协方差。

- `cross_decomposition`

包含CCA（典型相关分析）和PLS（偏最小二乘法）两种算法，用来探索两个多元数据集之间的线性关系。

- `datasets`

sklearn自带的数据采集器，主要功能是响应用户的调用，从网络下载toydata。

- `decomposition`

矩阵分解的工具包，包括主成分分析（PCA）、独立成分分析（ICA）和非负矩阵分解（NMF），其中PCA还有稀疏的实现。

- `ensemble`

集成学习的工具包，实现了各种bagging和boosting方法。

- `experimental`

实验模块。

- `externals`

外部依赖脚本。

- `feature_extraction`

特征提取，目前主要实现了从图片和文字中提取特征，以后`scikit-bio`扩展到从生物学数据如序列等中提取特征。

- `feature_selection`

特征选择，主要包括过滤方法（如根据方差）和递归特征删除算法。这个接口也可用于降维。

- `gaussian_process`

高斯过程，主要分为高斯分类和高斯回归。

- `impute`

缺失值填充，例如kNN算法。

- `inspection`

模型检查工具。

- `linear_model`

线型模型包，主要包括线性回归、逻辑斯蒂回归、LASSO、Ridge回归、感知机等等。

- `manifold`

流形学习包，主要包括Isomap、LLE和tSNE等。

- `metrics`

集合了大部分的性能评价指标，如准确性（accuracy）、auc、f1_score、hinge_loss、roc_auc_score和roc_curve。

- `mixture`

高斯混合模型和贝叶斯混合模型。

- `model_selection`

训练工具包，常见的有`GridSearchCV`、`TimeSeriesSplit`、`KFold`、`cross_validate`等。

- `neighbors`

K近邻算法包，主要包括球树、KD树、KNN分类、KNN回归等算法实现。

- `neural_network`

基础的神经网络模型，包括贝努利玻尔兹曼机、多层感知机分类和回归模型。

- `preprocessing`

预处理工具包，常见的有LabelEncoder、MinMaxScaler、Normalizer、OneHotEncoder等。

- `semi_supervises`

半监督学习算法包，如LabelPropagation、LabelSpreading等。

- `svm`

支持向量机算法包，包括线型svm分类/回归SVC/SVR、OneClassSVM等，复用了很多libsvm的包。

In [13]:
from sklearn.svm import SVC

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [15]:
SVC().get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

## 2. 关于`datasets`模块

`datasets`模块保存了一些实例数据集，并支持从头构造数据集。

In [2]:
from sklearn.datasets import make_classification

In [4]:
X, y = make_classification(n_samples=200, 
                           n_features=50,
                           n_informative=20, 
                           n_redundant=15,
                           n_classes=2,
                           weights = [0.25, 0.75],
                          random_state=1234)

### 2.1 参数解析
- `n_samples`：样本数量
- `n_features`：特征总数
- `n_informative`：有用的特征数
- `n_redundant`：无用的冗余的特征数
- `n_classes`：分类数目
- `weights`：不同分类的占比权重
- `random_state`：随机数生成种子
- `n_repeated`：重复特征数

In [5]:
print(X.shape)

(200, 50)


In [6]:
print(y.shape)

(200,)


In [10]:
for i in zip([1,2,3,4]):
    print(i)

(1,)
(2,)
(3,)
(4,)


In [13]:
for t in zip([1,2,3,4], {"A":3, "B":2, "C":2, "D":1}):
    print(t)

(1, 'A')
(2, 'B')
(3, 'C')
(4, 'D')


In [18]:
li = ["a", "b", "c"]
print("".join(li))

abc


In [23]:
x = [1,2,3.0]

In [24]:
assert(all([isinstance(i, int) for i in x]))

AssertionError: 

In [66]:
class IntList(list):
    
    def __init__(self, iterable):
        assert(all([isinstance(i, int) for i in iterable]))
        super(IntList, self).__init__()
        self.li = iterable
            
    def __mul__(self, num):
        for i in range(len(self.li)):
            self.li[i] *= num
        return IntList(self.li)
            
    def __str__(self):
        return str(self.li)

a = IntList([2,3,4])
a *= 3
print(a)

[6, 9, 12]


In [75]:
class MyList():
    def __init__(self):
        self.data = []
    def append(self, item):
        self.data.append(item)
    def __getitem__(self, key):
        print("key is : " + str(key))
        return self.data[key]

In [77]:
l = MyList()
l.append("my")
l.append("name")
l.append("is")
l.append("python")
l.append("cat")

l[:2]

key is : slice(None, 2, None)


['my', 'name']

In [83]:
x = range(10)
index = slice(0,6,1)
print(x.__getitem__(index))
print(x[0:6])

range(0, 6)
range(0, 6)


In [88]:
from collections.abc import Reversible

In [91]:
type(reversed(x))

range_iterator

In [96]:
x.__next__()

AttributeError: 'range' object has no attribute '__next__'